In [7]:
import os
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

# Define download directory
download_dir = os.path.abspath("PwC_PDFs")

# Ensure the directory exists
os.makedirs(download_dir, exist_ok=True)

# Configure Chrome options to handle PDF downloads
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,  # Set download location
    "download.prompt_for_download": False,  # Disable pop-ups
    "plugins.always_open_pdf_externally": True,  # Directly download PDFs instead of opening them
    "profile.default_content_settings.popups": 0,
})

# Set up WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

def download_pdf(link, title):
    try:
        driver.get(link)
        time.sleep(3)  # Wait for the page to load (adjust if necessary)

        # Find all links in the page
        pdf_links = driver.find_elements(By.TAG_NAME, "a")

        for pdf_link in pdf_links:
            href = pdf_link.get_attribute("href")

            if href and href.endswith(".pdf"):  # Ensure it's a PDF file
                print(f"Downloading: {href}")

                # Open the link in a new tab to trigger download
                driver.execute_script("window.open(arguments[0], '_blank');", href)
                time.sleep(5)  # Wait for download to start
                return

        print(f"No PDF found on page: {link}")

    except Exception as e:
        print(f"Error downloading PDF from {link}: {e}")

# Read links from CSV and process them
with open("pwc_india_insights_details.csv", "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        link = row["Link"]
        title = row["Title"].replace(" ", "_").replace("/", "-")  # Sanitize filename
        download_pdf(link, title)

# Close the browser after processing all links
driver.quit()

print(f"PDFs downloaded to {download_dir}")


No PDF found on page: https://www.pwc.in/ghost-templates/the-mutual-funds-route-to-viksit-bharat-2047.html
No PDF found on page: https://www.pwc.in/ghost-templates/financial-health-transcending-from-access-to-impact.html
No PDF found on page: https://www.pwc.in/ghost-templates/retail-reinvention-paradigm.html
No PDF found on page: https://www.pwc.in/ghost-templates/pwc-indias-financial-services-fs-risk-symposium-ministry-finance-keynote-session-february-2025.html
No PDF found on page: https://www.pwc.in/ghost-templates/india-spends-deep-dive-consumer-spending-behaviour.html
No PDF found on page: https://www.pwc.in/ghost-templates/future-quality-manufacturing.html
No PDF found on page: https://www.pwc.in/ghost-templates/powering-automation-with-agents.html
No PDF found on page: https://www.pwc.in/ghost-templates/deals-at-a-glance-annual-review-2024.html
PDFs downloaded to d:\personal\benori_assignment\pwc_india\PwC_PDFs
